In [1]:
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from keras.optimizers import RMSprop
import numpy as np
import random

In [2]:
!wget -O kafka_Metamorphosis.txt https://www.gutenberg.org/files/5200/5200-0.txt

--2023-04-06 06:11:00--  https://www.gutenberg.org/files/5200/5200-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 142017 (139K) [text/plain]
Saving to: ‘kafka_Metamorphosis.txt’

kafka_Metamorphosis 100%[===================>] 138.69K  --.-KB/s    in 0.1s    

2023-04-06 06:11:01 (1.29 MB/s) - ‘kafka_Metamorphosis.txt’ saved [142017/142017]



In [3]:
text = open('kafka_Metamorphosis.txt', 'r').read().lower()
print('Text lenght: ', len(text))

Text lenght:  138408


In [4]:
print(text[:1000])

﻿the project gutenberg ebook of metamorphosis, by franz kafka

this ebook is for the use of anyone anywhere in the united states and
most other parts of the world at no cost and with almost no restrictions
whatsoever. you may copy it, give it away or re-use it under the terms
of the project gutenberg license included with this ebook or online at
www.gutenberg.org. if you are not located in the united states, you
will have to check the laws of the country where you are located before
using this ebook.

** this is a copyrighted project gutenberg ebook, details below **
**     please follow the copyright guidelines in this file.     **

title: metamorphosis

author: franz kafka

translator: david wyllie

release date: may 13, 2002 [ebook #5200]
[most recently updated: may 20, 2012]

language: english

character set encoding: utf-8

copyright (c) 2002 by david wyllie.

*** start of the project gutenberg ebook metamorphosis ***




metamorphosis

by franz kafka

translated by david wyllie



In [5]:
chars = sorted(list(set(text)))
print('Total chars: ', len(chars))

Total chars:  62


In [6]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [7]:
maxlen = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])

print('nb sentences: ', len(sentences))

nb sentences:  46123


In [8]:
print(sentences[:3])
print(next_chars[:3])

['\ufeffthe project gutenberg ebook of metamorp', 'e project gutenberg ebook of metamorphos', 'roject gutenberg ebook of metamorphosis,']
['h', 'i', ' ']


In [9]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

<ipython-input-9-8b86d899b220>:1: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
<ipython-input-9-8b86d899b220>:2: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y = np.zeros((len(sentences), len(chars)), dtype=np.bool)


In [10]:
print(x[:3])
print(y[:3])

[[[False False False ... False False  True]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]]]
[[False False False False False False False False False False False False
  False False False False False False False False False False False False
  False False False False False Fals

In [11]:
model = Sequential()
model.add(LSTM(254, input_shape = (maxlen, len(chars))))
model.add(Dense(10*len(chars)))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))

In [12]:
model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.01))

/usr/local/lib/python3.9/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
model.fit(x, y, batch_size = 128, epochs = 10)

Epoch 1/10
361/361 [==============================] - 10s 8ms/step - loss: 3.6320
Epoch 2/10
361/361 [==============================] - 3s 8ms/step - loss: 1.9186
Epoch 3/10
361/361 [==============================] - 3s 9ms/step - loss: 1.6943
Epoch 4/10
361/361 [==============================] - 3s 8ms/step - loss: 1.5494
Epoch 5/10
361/361 [==============================] - 3s 8ms/step - loss: 1.4274
Epoch 6/10
361/361 [==============================] - 3s 8ms/step - loss: 1.3273
Epoch 7/10
361/361 [==============================] - 3s 8ms/step - loss: 1.2331
Epoch 8/10
361/361 [==============================] - 3s 8ms/step - loss: 1.1383
Epoch 9/10
361/361 [==============================] - 3s 8ms/step - loss: 1.0561
Epoch 10/10
361/361 [==============================] - 3s 8ms/step - loss: 0.9763


In [23]:
def sample(preds, temperature = 1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_text(sentence, length, diversity):
    generated = ''
    generated += sentence
    for i in range(length):
            x_pred = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(sentence):
                x_pred[0, t, char_indices[char]] = 1.

            preds = model.predict(x_pred, verbose=0)[0]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            generated += next_char
            sentence = sentence[1:] + next_char
    return generated

In [24]:
text = " The input is expected to have 40 chars "
len(text)

40

In [26]:
text = " The input is expected to have 40 chars "
sentence = text[0: maxlen]

print(generate_text(sentence, 30, 0.2))

KeyError: ignored